In [2]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import backtesting
backtesting.set_bokeh_output(notebook=False)
import yfinance as yf
import datetime as dt
import talib
import matplotlib.pyplot as plt

In [3]:
btc= yf.Ticker("BTC-USD")
btc_data = btc.history(start = dt.datetime(2015,1,1),
                       end=dt.datetime(2025,1,1), 
                       interval="1d")
btc_data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2015-01-01 00:00:00+00:00,320.434998,320.434998,314.002991,314.248993,8036550,0.0,0.0
2015-01-02 00:00:00+00:00,314.079010,315.838989,313.565002,315.032013,7860650,0.0,0.0
2015-01-03 00:00:00+00:00,314.846008,315.149994,281.082001,281.082001,33054400,0.0,0.0
2015-01-04 00:00:00+00:00,281.145996,287.230011,257.612000,264.195007,55629100,0.0,0.0
2015-01-05 00:00:00+00:00,265.084015,278.341003,265.084015,274.473999,43962800,0.0,0.0
...,...,...,...,...,...,...,...
2024-12-27 00:00:00+00:00,95704.976562,97294.843750,93310.742188,94164.859375,52419934565,0.0,0.0
2024-12-28 00:00:00+00:00,94160.187500,95525.898438,94014.289062,95163.929688,24107436185,0.0,0.0
2024-12-29 00:00:00+00:00,95174.054688,95174.875000,92881.789062,93530.226562,29635885267,0.0,0.0


In [5]:
class BTSMA(Strategy):
    n1 = 20
    n2 = 50

    def init(self):
        self.sma1 = self.I(talib.SMA, self.data.Close, self.n1)
        self.sma2 = self.I(talib.SMA, self.data.Close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()
        
btc_data = btc_data * 10**-6
bt = Backtest(btc_data, BTSMA, cash=1000, commission=.0025)
stats = bt.optimize(n1=range(1, 365,1), n2=range(1, 365,1), maximize='Equity Final [$]',
            constraint=lambda p: p.n1 < p.n2)

bt.plot()
print(stats)

: 